In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
import cv2 as cv
import os
from os import listdir
from os.path import isfile, join
from skimage.io import imread
from skimage.io import imshow
from skimage.color import rgb2gray
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import matplotlib.pyplot as plt
import numba as nb
import numpy as np

در قسمت زیر، مسیر ویدیوهای خوشحال و متعجب تعریف می شوند.

In [ ]:
dataPath1 = 'drive/MyDrive/CVLab/LAB1/Data/Cohn-Kanade/surprise videos'
dataPath2 = 'drive/MyDrive/CVLab/LAB1/Data/Cohn-Kanade/happy videos'

In [ ]:
# parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 5,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

تابع زیر به ازای بردار اندازه و زاویه ورودی، هیستوگرام زوایا را حساب می کند.

In [ ]:
@nb.jit
def hist_calculation(magnitude, angle):
  h = np.zeros(8)
  for mag, ang in zip(magnitude.reshape(-1), angle.reshape(-1)):
    if ang == 0:
      h[0] += mag
    elif ang > 0 and ang <= 360:
      h[int( (ang-1)//30) ] += mag
  return h

درقسمت زیر، ابتدا تصاویر مربوط به ویدیوهای متعحب و سپس تصاویر مربوط به ویدیوهای خوشحال خوانده می شوند. کامنت های گذاشته شده، عملکرد هر خط از کد را توضیح میدهند. تصویر هر فریم پس از خوانده شدن، به تصویر دوکاناله تبدیل می شود و روی آن ماسک (برای جدا کردن صورت شخص) اعمال می شود و ابعاد تصویر نهایی به دست آمده نصف می شود. سپس یک حلقه می زنیم که در آن شار نوری بین هر فریم جدیدی که خوانده می شود و فریم خوانده شده قبلی محاسبه می شود. اندازه و زوایای مربوط به فلو محاسبه شده و زاویه فلو به درجه تبدیل می شود. سپس با استفاده از تابع تعریف شده، هیستوگرام زوایای مربوط به فلو هر فریم محاسبه و ذخیره می شود. در انتهای لوپ، فریم قبلی با فریم فعلی جایگزین می شود. برای هر ویدیو، یکبار میانگین هیستوگرام زوایا محاسبه می شود و یکبار هم ماکسیمم آن ها.

In [ ]:
surprise_hists = []

for video in os.listdir(dataPath1):

  current_vid_hists = []

  dirs = os.listdir(join(dataPath1 , video))
  # reading the first frame
  prev_frame = imread(join(dataPath1 , video, dirs[0]))
  # converting the frame image to grayscale
  prev_frame_gray = rgb2gray(prev_frame)
  # masking
  prev_frame_gray = prev_frame_gray[100:700, 400:800]
  # resizing 
  prev_frame_gray = resize(prev_frame_gray, (0.5*prev_frame_gray.shape[0], 0.5*prev_frame_gray.shape[1]))
  for i in range(1, len(dirs)):
    # reading the frame
    current_frame = imread(join(dataPath1 , video, dirs[1]))
    # converting the frame image to grayscale
    current_frame_gray = rgb2gray(current_frame)
    # masking the image
    current_frame_gray = current_frame_gray[100:700, 400:800]
    # resizing the image
    current_frame_gray = resize(current_frame_gray, (0.5*current_frame_gray.shape[0], 0.5*current_frame_gray.shape[1]))
    
    # calculating optical flow
    flow = cv.calcOpticalFlowFarneback(prev_frame_gray, current_frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # computes the magnitude and angle of the flow
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
    angle = angle * 180 / np.pi / 2

    # computing histogram
    h = hist_calculation(magnitude, angle)
    
    current_vid_hists.append(h)

    prev_frame_gray = current_frame_gray

  current_vid_hists = np.array(current_vid_hists)

  max_hist = np.max(current_vid_hists, axis=0)
  mean_hist = np.mean(current_vid_hists, axis=0)
  hist = list(max_hist) + list(mean_hist)

  surprise_hists.append(hist)

In [ ]:
happy_hists = []

for video in os.listdir(dataPath2):

  current_vid_hists = []

  dirs = os.listdir(join(dataPath2 , video))
  # reading the first frame
  prev_frame = imread(join(dataPath2 , video, dirs[0]))
  # converting the frame image to grayscale
  prev_frame_gray = rgb2gray(prev_frame)
  # masking
  prev_frame_gray = prev_frame_gray[100:700, 400:800]
  # resizing 
  prev_frame_gray = resize(prev_frame_gray, (0.5*prev_frame_gray.shape[0], 0.5*prev_frame_gray.shape[1]))
  for i in range(1, len(dirs)):
    # reading the frame
    current_frame = imread(join(dataPath2 , video, dirs[1]))
    # converting the frame image to grayscale
    current_frame_gray = rgb2gray(current_frame)
    # masking the image
    current_frame_gray = current_frame_gray[100:700, 400:800]
    # resizing the image
    current_frame_gray = resize(current_frame_gray, (0.5*current_frame_gray.shape[0], 0.5*current_frame_gray.shape[1]))
    
    # calculating optical flow
    flow = cv.calcOpticalFlowFarneback(prev_frame_gray, current_frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # computes the magnitude and angle of the flow
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
    angle = angle * 180 / np.pi / 2
    
    # computing histogram
    h = hist_calculation(magnitude, angle)
    
    current_vid_hists.append(h)

    prev_frame_gray = current_frame_gray

  current_vid_hists = np.array(current_vid_hists)

  max_hist = np.max(current_vid_hists, axis=0)
  min_hist = np.mean(current_vid_hists, axis=0)
  hist = list(max_hist) + list(min_hist)

  happy_hists.append(hist)

In [ ]:
happy_labels = list( np.zeros(30) )
surprise_labels = list ( np.ones(30) )

در قسمت زیر، نمونه های به دست آمده به نمونه های آموزشی و تست تقسیم می شوند.

In [60]:
# splitting data to train and test
X_train_happy, X_test_happy, y_train_happy, y_test_happy = train_test_split(happy_hists, happy_labels, test_size=0.3, random_state=42)
X_train_surprise, X_test_surprise, y_train_surprise, y_test_surprise = train_test_split(surprise_hists, surprise_labels, test_size=0.3, random_state=42)

In [61]:
X_train = X_train_happy + X_train_surprise
y_train = y_train_happy + y_train_surprise
X_test = X_test_happy + X_test_surprise
y_test = y_test_happy + y_test_surprise

در قسمت زیر، با استفاده از طبقه بند اس وی ام خطی، نمونه ها طبقه بندی می شوند. با تغییر پارامترها، بهترین مقداری که برای نرخ تست به دست آمد گزارش شده است.

In [83]:
clf1 = svm.LinearSVC(C=0.01)
clf1.fit(X_train, y_train)

LinearSVC(C=0.01)

In [84]:
y_train_predicted1 = clf1.predict(X_train)

In [85]:
y_test_predicted1 = clf1.predict(X_test)

In [87]:
test_acc1 = accuracy_score(y_test, y_test_predicted1)
print('Accuracy on the test data: ', test_acc1)

Accuracy on the test data:  0.6666666666666666
